In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
features_path='./01-data/06-features/'
sum=0
for slide in os.listdir(features_path):
    sum+=len(os.listdir(os.path.join(features_path,slide)))
# 6042668

In [ ]:
#select random patches
import random
def reservoir_sampling(root_dir, k=10000):
    reservoir = []  
    count = 0      

    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            filepath = os.path.join(dirpath, filename)
            count += 1
            if len(reservoir) < k:
                reservoir.append(filepath)
            else:
                j = random.randint(0, count - 1)
                if j < k:
                    reservoir[j] = filepath
    return reservoir

root_dir = './01-data/06-features/'  
selected_files = reservoir_sampling(root_dir, k=10000)
print(f"随机抽取的文件数：{len(selected_files)}")
for file in selected_files[:10]:
    print(file)

随机抽取的文件数：10000
/2data/liyixin/03-PPS与空间基因相关性分析/01-data/06-features/TCGA-DD-A73B-01Z-00-DX2/patch_2137.json
/2data/liyixin/03-PPS与空间基因相关性分析/01-data/06-features/TCGA-RC-A7SH-01Z-00-DX1/patch_951.json
/2data/liyixin/03-PPS与空间基因相关性分析/01-data/06-features/TCGA-DD-AACC-01Z-00-DX1/patch_14069.json
/2data/liyixin/03-PPS与空间基因相关性分析/01-data/06-features/TCGA-DD-A116-01Z-00-DX1/patch_23230.json
/2data/liyixin/03-PPS与空间基因相关性分析/01-data/06-features/TCGA-G3-A5SL-01Z-00-DX1/patch_12855.json
/2data/liyixin/03-PPS与空间基因相关性分析/01-data/06-features/TCGA-DD-A4ND-01Z-00-DX1/patch_9093.json
/2data/liyixin/03-PPS与空间基因相关性分析/01-data/06-features/TCGA-FV-A4ZP-01Z-00-DX1/patch_1531.json
/2data/liyixin/03-PPS与空间基因相关性分析/01-data/06-features/TCGA-DD-A1EC-01Z-00-DX1/patch_2035.json
/2data/liyixin/03-PPS与空间基因相关性分析/01-data/06-features/TCGA-MI-A75E-01Z-00-DX2/patch_25564.json
/2data/liyixin/03-PPS与空间基因相关性分析/01-data/06-features/TCGA-ED-A8O5-01Z-00-DX1/patch_19290.json


In [ ]:
import concurrent.futures
import json
key_to_extract = 'exp'  

def extract_vector(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        vector = data.get(key_to_extract)
        if isinstance(vector, list) and len(vector) == 183:
            return vector
        else:
            print(f"文件 {file_path} 中键 '{key_to_extract}' 不存在或其值不是长度为183的列表。")
            return None
    except Exception as e:
        print(f"处理文件 {file_path} 时出错: {e}")
        return None

results = []
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    for vector in executor.map(extract_vector, selected_files):
        if vector is not None:
            results.append(vector)

matrix = np.array(results)
print("矩阵形状:", matrix.shape)

矩阵形状: (10000, 183)


In [ ]:
#find gene_idx in gene_list
sig_genes=['DES',
'FOXP3',
'CD19',
'IL1RL1',
'PDPN',
'RERGL',
'ANGPT2',
'ALDH1A3',
'MYH11',
'MALL',
'THY1',
'SFRP4',
'APOLD1',
'LYVE1',
'RNASE1',
'CD34',
'CD93',
'ADAMTS1',
'FBN1',
'CFHR1',
'CAVIN1',
'NOP53',
'EPAS1',
'RHOA',
'BTF3',
'GNAS']

gene_list_path = './01-data/04-Liver/liver_hvg_cut_200_minus3.npy'
gene_list = list(np.load(gene_list_path))
sig_gene_index=[gene_list.index(i) for i in sig_genes]
sig_gene_index

[59,
 81,
 26,
 105,
 143,
 150,
 7,
 5,
 134,
 123,
 173,
 159,
 8,
 122,
 153,
 32,
 40,
 3,
 72,
 42,
 20,
 139,
 68,
 152,
 13,
 87]

In [ ]:
#find binary threshold by GMM
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture

fig, axes = plt.subplots(6, 5, figsize=(20, 20))
axes = axes.flatten()
thresholds={}
for i, col in enumerate(sig_gene_index):
    sorted_feature = np.sort(matrix[:, col])
    axes[i].plot(sorted_feature, marker='o', linestyle='-', color='b')
    sorted_feature=sorted_feature.reshape(-1,1)
    gmm = GaussianMixture(n_components=2, covariance_type='full')
    gmm.fit(sorted_feature)
    means = gmm.means_.flatten()

    threshold = np.mean(means)
    axes[i].axhline(y=threshold, color='red', linestyle='--', label=f'Threshold = {threshold:.2f}')
    
    gene_name = gene_list[col] 
    thresholds[gene_name]=threshold
    axes[i].set_title(f'${gene_name}$', fontsize=18) 
    axes[i].grid(True)
    axes[i].legend()

for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
# plt.show()
plt.savefig("./03-results/02-threshold/05-GMM_threshold.png", dpi=300, bbox_inches='tight')
plt.close()


In [ ]:
#保存用GMM自动选择的阈值结果
with open('./03-results/02-threshold//05-GMM-autoselect-thresholds.json','w') as f:
    json.dump(thresholds,f)